# Differentiable Fluid Simulations with Φ<sub>Flow</sub>

This notebook steps you through setting up fluid simulations and using TensorFlow's differentiation to optimize them.

Execute the cell below to install the [Φ<sub>Flow</sub> Python package from GitHub](https://github.com/tum-pbs/PhiFlow).

In [ ]:
!pip install --upgrade --quiet phiflow

In [ ]:
from phi.flow import *  # The Dash GUI is not supported on Google Colab, ignore the warning
import pylab

# Setting up a simulation

Φ<sub>Flow</sub> is object-oriented, i.e. you assemble your simulation by constructing a number of objects and adding them to the world.

The following code sets up four fluid simulations that run in parallel (`batch_size=4`). Each fluid simulation has a circular Inflow at a different location.

In [ ]:
world = World()
fluid = world.add(Fluid(Domain([40, 32], boundaries=CLOSED), buoyancy_factor=0.1, batch_size=4), physics=IncompressibleFlow())
world.add(Inflow(Sphere(center=[[5,4], [5,8], [5,12], [5,16]], radius=3), rate=0.2));

The inflow affects the fluid's marker density. Because the `buoyancy_factor` is positive, the marker creates an upward force.

Let's plot the marker density after one simulation frame.

In [ ]:
world.step()
pylab.imshow(np.concatenate(fluid.density.data[...,0], axis=1), origin='lower', cmap='magma')

We can run more steps by repeatedly calling `world.step()`.

In [ ]:
for frame in range(20):
  print('Computing frame %d' % frame)
  world.step(dt=1.5)

In [ ]:
pylab.imshow(np.concatenate(fluid.density.data[...,0], axis=1), origin='lower', cmap='magma')

# Differentiation

The simulation we just computed was using purely NumPy (non-differentiable) operations.
To enable differentiability, we need to build a TensorFlow graph that computes this result.

In [ ]:
%tensorflow_version 1.x
from phi.tf.flow import *  # Causes deprecation warnings with TF 1.15
import pylab
session = Session(None)  # Used to run the TensorFlow graph

Let's set up the simulation just like before. But now, we want to optimize the initial velocities so that all simulations arrive at a final state that is similar to the first simulation from the previous example. I.e., the state shown in the left-most image above.

To achieve this, we create a TensorFlow variable for the velocity at t=0.
It is initialized with zeros (like with the NumPy simulation above) and can later be used as a target for optimization.

In [ ]:
world = World()
fluid = world.add(Fluid(Domain([40, 32], boundaries=CLOSED), buoyancy_factor=0.1, batch_size=4), physics=IncompressibleFlow())
world.add(Inflow(Sphere(center=[[5,4], [5,8], [5,12], [5,16]], radius=3), rate=0.2));
fluid.velocity = variable(fluid.velocity)  # create TensorFlow variable
initial_state = fluid.state  # Remember the state at t=0 for later visualization
session.initialize_variables()



Note that we actually created two variables, one for each velocity component. If you're interested in how this magic works, have a look at the [Struct documentation](https://github.com/tum-pbs/PhiFlow/blob/master/documentation/Structs.ipynb).

In [ ]:
[print(grid.data) for grid in fluid.velocity.unstack()];

If you look closely, you'll notice that the shapes of the variables differ. This is because the velocity is sampled in [staggered form](https://github.com/tum-pbs/PhiFlow/blob/master/documentation/Staggered_Grids.md).

The simulation now contains variables in the initial state.
Since all later states depend on the value of the variable, the `step` method cannot directly compute concrete state values.
Instead, `world.step` will extend the TensorFlow graph by the operations needed to perform the step.

To execute the graph with actual data, we can use `session.run`, just like with regular TensorFlow 1.x. While `run` would usually be used to infer predictions from a learning model, it now executes the graph of simulation steps.

In [ ]:
world.step()
pylab.imshow(np.concatenate(session.run(fluid.density).data[...,0], axis=1), origin='lower', cmap='magma')

Let's build a graph for the full simulation.

In [ ]:
for frame in range(20):
  print('Building graph for frame %d' % frame)
  world.step(dt=1.5)

When calling `session.run` now, the full simulation is evaluated using TensorFlow operations.
This will take advantage of your GPU, if available.
If you compile Φ<sub>Flow</sub> with [CUDA support](https://github.com/tum-pbs/PhiFlow/blob/master/documentation/Installation_Instructions.md), the TensorFlow graph will use optimized operators for efficient simulation and training runs.

In [ ]:
print('Computing frames...')
pylab.imshow(np.concatenate(session.run(fluid.density).data[...,0], axis=1), origin='lower', cmap='magma')

Next, we define the *loss* function (also called *cost* or *objective* function). This is the value we want to decrease via optimization.
For this example, we want the marker densities of all final simulation states to match the left-most one, called `target`.

For the optimizer, we choose gradient descent for this example.

In [ ]:
target = session.run(fluid.density).data[0,...]
loss = math.l2_loss(fluid.density.data[1:,...] - target)
optim = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)
session.initialize_variables()
print('Initial loss: %f' % session.run(loss))

With the loss and optimizer set up, all that's left is to run the actual optimization.

In [ ]:
for optim_step in range(10):
  print('Running optimization step %d. %s' % (optim_step, '' if optim_step else 'The first step sets up the adjoint graph.'))
  _, loss_value = session.run([optim, loss])
  print('Loss: %f' % loss_value)

In [ ]:
pylab.imshow(np.concatenate(session.run(fluid.density).data[...,0], axis=1), origin='lower', cmap='magma')

Now that the optimization has done its work, we can have a look at the now-optimized initial velocity field.

In [ ]:
optimized_velocity_field = session.run(initial_state.velocity).at_centers()

In [ ]:
pylab.title('Initial y-velocity (optimized)')
pylab.imshow(np.concatenate(optimized_velocity_field.data[...,0], axis=1), origin='lower')

In [ ]:
pylab.title('Initial x-velocity (optimized)')
pylab.imshow(np.concatenate(optimized_velocity_field.data[...,1], axis=1), origin='lower')

This notebook provided an introduction to running fluid simulations in NumPy and TensorFlow.
It demonstrated how to use the gradients provided by Φ<sub>Flow</sub> to run simple optimizations over the course of several timesteps.

For additional examples, e.g. coupling simulations with neural networks, please check the [other demos](https://github.com/tum-pbs/PhiFlow/tree/master/demos).